In [2]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import pickle
import matplotlib.pyplot as plt

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------
np.set_printoptions(precision=8, linewidth = 75)

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


# Pose Graph Optimization

### GOAL: Adjust configuration of absolute poses (nodes) to minimize squared error intorduced by constraints (edges)


Node: [x, y, z, r, p, y]

Edge: Odometry Measurements

# State vector $x$

absolute position and orientation of vehicle at each timestep

\begin{equation}
\textbf{x} = 
\begin{bmatrix}
\textbf{x}_1\\
\textbf{x}_2\\
\textbf{x}_3 \\
\vdots
\end{bmatrix} = 
\begin{bmatrix}
x_1 & y_1 & z_1 & r_1 & p_1 & y_1 \\
x_2 & y_2 & z_2 & r_2 & p_2 & y_2 \\
x_3 & y_3 & z_3 & r_3 & p_3 & y_3 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \\ 
\end{bmatrix}
\end{equation}

# Transformations $\mathbf{X}$  
Represeneted in Homogenous Coordinates. This is an overparameterization (similar to using quaternions) which helps avoid singularities when calculating the error function to adjust each Gauss-Newton iteration, however, the additional degrees of freedom need to be removed before forming our state vector

\begin{equation}
\mathbf{X_i} = 
\begin{bmatrix}
R_{11} & R_{12} & R_{13} & dx\\
R_{21} & R_{22} & R_{23} & dy \\
R_{31} & R_{32} & R_{33} & dz \\
0 & 0 & 0 & 1
\end{bmatrix} 
\end{equation}


$\big(\mathbf{X}_i^{-1} \mathbf{X}_{i+1} \big)$  describes how node $i$ sees node $(i+1)$ (ex: odometry)

$\big(\mathbf{X}_i^{-1} \mathbf{X}_{j} \big)$  describes how node $i$ sees node $j$ (ex: loop closure)


<span style="color:red"> Important note: $X$ will need to contain more elements than $x$, since $X$ must represent all state pairs observed in constraints $Z$, while $x$ only needs a single element to represent the absolute pose of each state </span>


In [3]:
def v2t(vector):
    """converts a transformation vector to homogenous coordinate system"""
    if len(tf.shape(vector)) == 1: #allow for 1-D or N-D input
        vector = vector[None,:]
    angs = vector[:,3:]
    phi = angs[:,0]
    theta = angs[:,1]
    psi = angs[:,2]
    rot = tf.Variable([[cos(theta)*cos(psi), sin(psi)*cos(phi) + sin(phi)*sin(theta)*cos(psi), sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi)],
                       [-sin(psi)*cos(theta), cos(phi)*cos(psi) - sin(phi)*sin(theta)*sin(psi), sin(phi)*cos(psi) + sin(theta)*sin(psi)*cos(phi)],
                       [sin(theta), -sin(phi)*cos(theta), cos(phi)*cos(theta)]])
    rot = tf.transpose(rot, [2, 0, 1])
    trans = vector[:,:3]
    trans = np.reshape(trans, (np.shape(rot)[0], 3, 1))
    transform = tf.concat((rot, trans), axis = -1)
    extra = tf.tile(tf.constant([[[0., 0., 0., 1.]]], dtype = tf.double), (np.shape(rot)[0],1,1))
    transform = tf.concat((transform, extra), axis = -2)
    return transform

In [4]:
def t2v(mat):
    """converts transformation matrix to state vector"""
    if len( tf.shape(mat) ) == 2:
        mat = mat[None, :, :]
    R_sum = np.sqrt(( mat[:,0,0]**2 + mat[:,0,1]**2 + mat[:,1,2]**2 + mat[:,2,2]**2 ) / 2)
    phi = np.arctan2(-mat[:,1,2],mat[:,2,2])
    theta = np.arctan2(mat[:,0,2], R_sum)
    psi = np.arctan2(-mat[:,0,1], mat[:,0,0])
    angs = np.array([phi, theta, psi])
    vector = tf.concat((mat[:,:3,-1], angs.T), axis =1)
    return vector

In [ ]:
#NOTE: v2t() -> t2v() is returning to the same value only if I take inverse (for rotation) 
#                     and without inverse only (for translation). [T]^-1 @ [T] = I still
#Question: is this normal?  

# # test = np.array([1., 2., 3., 0.05, 0.00, -0.14])
# test = np.ones([2,6])
# test[1,:] = np.array([1., 2., 3., 0.003, 0.001, 0.3])
# T = v2t(test)
# # print("T: \n",T)
# vect_trans = t2v(T).numpy()[:,:3]
# vect_rot = t2v(tf.linalg.inv(T)).numpy()[:,3:]
# print(np.append(vect_trans, vect_rot, axis = 1))

# Least Squares Error Function


The optimial state vector, $x^*$, occurs where:

\begin{equation}
\Large
x^* = \arg\min_x \sum_{ij}^{} e^T_{ij}(x_i,x_j)\Omega_{ij}e_{ij}(x_i,x_j)
\end{equation}

$\Omega_{ij}$ is the information matrix associated with the odometry estimate that relates $i$ and $j$. $\Omega_{ij}$ is the inverse of the covariance matrix $\sigma_{ij}$

the error funcion for each connected node $i,j$ as a function of the state vector $x$ is defined as:

\begin{equation}
\Large
e_{ij}(x) = \text{t2v}(Z_{ij}^{-1} (X_j^{-1}X_i))
\end{equation}

\begin{equation}
Z_{ij}^{-1} = \text{constraint (from measurement)}
\end{equation}

\begin{equation}
(X_j^{-1}X_i) = x_i \text{ relative to }x_j \text{ given the current model of system} 
\end{equation}

$ \text{Stachniss and Grisetti use} \boxplus \text{to repersent the mapping of euclidian space } v \text{ to a homogenous coordinate frame } t$


Important Note: $e_{ij}(x)$ only depends on $x_i$ and $x_j$ 

In [237]:
def get_e(Zij, Xij):
    """calculates error function w.r.t. nodes i and j
    Zij == pose j relative to i according to nodes (rotation matrix)
    Xij == pose j relative to i according to constraints (rotation matrix)
    """        
    e = t2v(tf.linalg.pinv(Zij) @ Xij)

    return(e)    

In [592]:
# Zij = v2t(np.array([1.,  1.9, 0., 0.0003, 0.0005, 0.01 ])) # get from graph 
# Xij = v2t(np.array([1.1, 2.0, 0., 0.,     0.0,    0.008])) # get from odometry message
Zij =  v2t(np.zeros([2,6]))
Xij =  v2t(np.random.randn(2,6))
ij = np.array([[0,1],[1,2]])

e = get_e(Zij, Xij)
# print(e)

a, b = get_A_ij_B_ij(e)
# print(a)

J = get_J(e, ij)
# print(J[1])

# Linearizing the System

\begin{equation}
\Large
e_{ij}(x + \Delta x) \approx e_{ij}(x) + J_{ij} \Delta x
\end{equation}

Here, $J_{ij}$ is the the jacobian of $e_{ij}$ with respect to x

\begin{equation}
\Large
J_{ij} = \frac{\delta e_{ij}(x)}{\delta x} = \bigg{(} 0 \dots \frac{\delta e_{ij}(x_i)}{\delta x_i}
\dots \frac{\delta e_{ij}(x_j)}{\delta x_j} \dots 0 \bigg{)} = \bigg{(} 0 \dots A_{ij} \dots B_{ij} \dots 0 \bigg{)} 
\end{equation}

\begin{equation}
A_{ij}, B_{ij} \in \mathbb{R}^{6 \times 6}
\end{equation}

\begin{equation}
J_{ij}(x) \in \mathbb{R}^{6 \times 6N}
\end{equation}
where $N$ is the total number of poses being solved for (nodes)

In [721]:
def get_A_ij_B_ij(e_ij):
    """calculates nonzero terms from the Jacobian of error function w.r.t. nodes i and j using TensorFlow
        e_ij == error function [x, y, z, phi, theta, psi]
        
        NOTE: this works with batch operations: error vectors passed in as tensor will result in
                corresponding output in the same order 
    """
    e_ij = tf.cast(e_ij, tf.float32)
    
    p_point = e_ij[:,:3]
    phi = e_ij[:,3][:,None]
    theta = e_ij[:,4][:,None]
    psi = e_ij[:,5][:,None]
    
    eyes = tf.tile(-tf.eye(3)[None,:,:], [tf.shape(p_point)[0] , 1, 1])
    
#     deriv of R() wrt phi
    dRdPhi = tf.Variable([[tf.zeros(len(phi), dtype = phi.dtype)[:,None], (-sin(psi)*sin(phi) + cos(phi)*sin(theta)*cos(psi)), (cos(phi)*sin(psi) + sin(theta)*sin(phi)*cos(psi))],
                       [tf.zeros(len(phi), dtype = phi.dtype)[:,None], (-sin(phi)*cos(psi) - cos(phi)*sin(theta)*sin(psi)), (cos(phi)*cos(psi) - sin(theta)*sin(psi)*sin(phi))], 
                       [tf.zeros(len(phi), dtype = phi.dtype)[:,None], (-cos(phi)*cos(theta)), (-sin(phi)*cos(theta))] ])[:,:,:,0]
    dRdPhi = tf.transpose(dRdPhi, (2,0,1))
    Jx = dRdPhi @ p_point[:,:,None]
    
    # (deriv of R() wrt theta).dot(p_point)
    dRdTheta = tf.Variable([[(-sin(theta)*cos(psi)), (cos(theta)*sin(phi)*cos(psi)), (-cos(theta)*cos(phi)*cos(psi))],
                               [(sin(psi)*sin(theta)), (-cos(theta)*sin(phi)*sin(psi)), (cos(theta)*sin(psi)*cos(phi))],
                               [(cos(theta)), (sin(phi)*sin(theta)), (-sin(theta)*cos(phi))] ])[:,:,:,0]
    dRdTheta = tf.transpose(dRdTheta, (2,0,1))
    Jy = dRdTheta @ p_point[:,:,None]

    # deriv of R() wrt psi
    dRdPsi = tf.Variable([[(-cos(theta)*sin(psi)), (cos(psi)*cos(phi) - sin(phi)*sin(theta)*sin(psi)), (cos(psi)*sin(phi) + sin(theta)*cos(phi)*sin(psi)) ],
                                       [(-cos(psi)*cos(theta)), (-sin(psi)*cos(phi) - sin(phi)*sin(theta)*cos(psi)), (-sin(phi)*sin(psi) + sin(theta)*cos(psi)*cos(phi))],
                                       [tf.zeros(len(phi), dtype = phi.dtype)[:,None],tf.zeros(len(phi), dtype = phi.dtype)[:,None],tf.zeros(len(phi), dtype = phi.dtype)[:,None]]])[:,:,:,0]
    dRdPsi = tf.transpose(dRdPsi, (2,0,1))
    Jz = dRdPsi @ p_point[:,:,None]
    
    top = tf.concat([eyes, Jx, Jy, Jz], axis = 2)
    flipped = tf.concat([tf.transpose(Jx, (0,2,1)), 
                         tf.transpose(Jy, (0,2,1)), 
                         tf.transpose(Jz, (0,2,1))], axis = 1)
    bottom = tf.concat([flipped, eyes], axis = 2) #was this
#     bottom = tf.concat([-flipped, -eyes], axis = 2) #test
    A_ij = tf.concat([top, bottom], axis = 1) #was this
#     A_ij = -tf.concat([top, bottom], axis = 1) #test
    B_ij = -A_ij #was this
#     B_ij = A_ij #test


    return A_ij, B_ij

In [708]:
def get_J(e, ij):
    """Forms sparse jacobian J, with entries A and B at indices i and j 
    J == [N, 6, 6*N], N = number of nodes
    e == [N, 6] error vectors
    ij == [N, 2] contains scan indices of each node ex: [scan2, scan5]
    """
    total_num_of_nodes = np.max(ij) + 1 #TODO: is this too big??
    if len(tf.shape(ij))< 2: #expand dimensions if only single pair passed in
        ij = ij[None,:]
    A_ij, B_ij = get_A_ij_B_ij(e)
    
    # Need to tile DIFFERENT AMOUNT depending on the index 
    #    TODO: move to batch operation?
    J = tf.zeros([0,6,total_num_of_nodes*6])
    for k in range(len(ij)):
        #TODO: add logic for when i and j are same value, when j = i + 1 ...
        leading = tf.tile(tf.zeros([6,6]), [1, ij[k,0] ]) #leading zeros before i
#         print("\n leading \n", leading)
        between = tf.tile(tf.zeros([6,6]), [1, ij[k,1] - ij[k,0] - 1 ]) #zeros between i and j
#         print("\n between: \n", between)
        ending  = tf.tile(tf.zeros([6,6]), [1, total_num_of_nodes - ij[k,1] - 1 ])
        J_ij = tf.concat([leading, A_ij[k], between, B_ij[k], ending], axis = 1)
        J = tf.concat((J, J_ij[None,:,:]), axis = 0)

    return J

In [709]:
# # ij = tf.constant(odometry_history[1:3,6:].astype(int))
# ij = np.array([[1,3],
#                [0,2]])
# # print(ij)
# # print(np.max(ij))
# # print("e: \n", e)

# J = get_J(e, ij)  
# # print("\n J \n",J[0])
# # print("\n J \n",tf.shape(J),  "\n", np.around(J.numpy(), 3))

# Accumumlate Sparse Tensors

calculate jacobians in batch operation first, then bring to sparse tensor for matmul??

\begin{equation}
\Large
b^T = \sum_{ij}^{}b_{ij} = \sum_{ij}^{}e_{ij}^T \Omega_{ij} J_{ij}
\end{equation}

In [710]:
def get_information_matrix(pred_stds):
    """returns information matrix (omega) from ICET cov_estimates"""
    pred_stds = tf.convert_to_tensor(pred_stds)[:,:,None] #convert to TF Tensor
    cov = pred_stds @ tf.transpose(pred_stds, (0,2,1))    #convert predicted stds -> covariance matrix
    info = tf.linalg.pinv(cov) #invert
    return info

In [711]:
def get_b(e, omega, J):
    """gets b matrix, using batched operations """
    b_T = tf.math.reduce_sum(tf.transpose(e, (0,2,1)) @ omega @ tf.cast(J, tf.double), axis = 0)
#     print("\n b_T: \n", tf.shape(b_T))
    b = tf.transpose(b_T)
    return b

In [712]:
def get_ij(ij_raw):
    """generates ij matrix, which describes which nodes are connected to 
       each other through odometry constraints. 
       Removes skipped indices and shifts everything to start at 0"""
#     print("ij_raw: \n", ij_raw)    
    y, idx = tf.unique(tf.reshape(ij_raw, [-1]))
#     print("\n y: \n", y, "\n idx: \n", idx)    
    ij = tf.reshape(idx, [-1,2])
    return ij

\begin{equation}
\Large
H = \sum_{ij}^{}H_{ij} = \sum_{ij}^{}J_{ij}^T \Omega J_{ij}
\end{equation}

Init Hessian once at start of simulation, update in place as new linearizations are added(?)

### Need to apply constraint to first node in kinematic chain

\begin{equation}
\Large
H_{11} \rightarrow H_{11} + I
\end{equation}

Optimization routine can't make sense of only relative measurements. We need to specify the first pose as the origin

In [713]:
def get_H(J, omega):
    """returns hessian H"""
    H_ij = tf.transpose(J, (0,2,1)) @ tf.cast(omega, tf.float32) @ J
    H = tf.math.reduce_sum(H_ij, axis = 0)
    constrain_11 = tf.pad(tf.eye(6), [[0,len(H)-6],[0,len(H)-6]])
    H = H + constrain_11    
    
#     H = tf.convert_to_tensor(np.tril(H.numpy()).T + np.tril(H.numpy(),-1)) #forces H to be symmetric
    
    return H

# Convert between $X$ and $x$

\begin{equation}
\Large
X \in \mathbb{R}^{M \times 4 \times 4}
\end{equation}

Where $M$ is the total number of constraints

\begin{equation}
\Large
x \in \mathbb{R}^{N \times 6}
\end{equation}

$N$ is the total number of nodes being optimized for

In [725]:
def get_X(x, ij):
    """given x, a list of global poses, this function returns 
       the relative transformations X, that describe the same relationships described by the constraints z

       x  -> global poses of each transpose
       ij -> indices of first and second element of x being considered
       """
    #get transform of fist elements in each pair, ordered by how they appear in ij
    first_vec = tf.gather(x, ij[:,0])
    second_vec = tf.gather(x, ij[:,1])
    difference = first_vec - second_vec
    #linearize differnce using euler angles
    X = v2t(tf.cast(difference, tf.double))    
    return X

# X_test = get_X(x, ij)
# # print(X_test)

# Solve the Linear System 

\begin{equation}
\Large
H \Delta x = -b
\end{equation}

In [723]:
odometry_history = np.load("test_data/leddartech_pixset/T_vec_history.npy")[5:15,:] #Load test data
# print(odometry_history)
pred_stds_history = np.load("test_data/leddartech_pixset/cov_vec_history.npy")[5:15,:]
ij = get_ij(odometry_history[:,6:].astype(np.int32))

# constraints (from odometry measurements)
Z =  v2t(odometry_history[:,:6])      
# relative pose estimates (from initial configuration of graph)
X =  v2t(odometry_history[:,:6] + 0.01*np.ones(np.shape(odometry_history[:,:6])))
# X =  v2t(odometry_history[:,:6]*0)

e = get_e(Z, X)
omega = get_information_matrix(pred_stds_history[:,:6])
J = get_J(e, ij)
b = get_b(e[:,:,None], omega, J)
H = get_H(J, omega)
# print("ij: \n", ij)
# print("e: \n", tf.shape(e))
# print("J: \n", tf.shape(J))
# print("omega: \n", tf.shape(omega))
# print("b: \n", tf.shape(b))
# print("H: \n", tf.shape(H))
# print(tf.linalg.eigvals(H)) # negative?

In [724]:
#solve the naive way

#get constraints
Z =  v2t(odometry_history[:,:6])   # [N, 6] -> [N, 4, 4]   
x = tf.zeros([tf.math.reduce_max(ij)+1,6])

for count in range(4):
    #create linear system----------------------------
    #update transformation matrix using ground new ground truth solution vector
    X = get_X(x, ij)
    e = get_e(Z, X)
    omega = get_information_matrix(pred_stds_history[:,:6])
    J = get_J(e, ij)
    b = get_b(e[:,:,None], omega, J)
    H = get_H(J, omega)
    #------------------------------------------------
    
    #solve linear system-----------------------------
    delta_x = -tf.linalg.pinv(H) @ tf.cast(b, tf.float32)
    #delta_x is a [N*6, 1] vector, need to reshape to same dimensions as x
    delta_x = tf.reshape(delta_x, (-1, 6))
    print("\n delta_x: \n:",delta_x.numpy())
    #------------------------------------------------
    
    #update solution vector--------------------------
    x = x - delta_x
    #------------------------------------------------
    
print(x)    


 delta_x: 
: [[-1.6145874  -0.81245613 -0.23935112 -0.14154446 -0.09594119  0.70031536]
 [-1.0618135  -0.5780418  -0.04721653  0.03541899 -0.07807732  0.46795392]
 [-0.3786347  -0.4486642   0.01295486 -0.06655431  0.09134698  0.34683454]
 [-0.12254283  0.11549784  0.03169253  0.03910764 -0.00879556 -0.08033103]
 [ 0.59270376  0.22626138  0.0327298   0.04057258 -0.01671172 -0.19570169]
 [ 0.48195875  0.32983452  0.05709227  0.05637245 -0.00986033 -0.2618348 ]
 [ 1.0249753   0.6007707   0.07723802  0.00511963  0.0772149  -0.506984  ]
 [ 1.0778832   0.56682897  0.07481524  0.03134977  0.04095789 -0.47026747]]

 delta_x: 
: [[-1.8456135  -0.8590379  -0.28690743 -0.12404335  0.03670931  0.16906762]
 [-0.9902344   0.41309357 -0.14705086  0.55947304 -0.45311356  0.19176626]
 [ 0.3893242  -0.8720169   0.09252739 -0.140841   -0.29574108 -1.5153847 ]
 [-0.5765457   0.13772202  0.13005066 -0.33589816  0.59791183  1.7388077 ]
 [-0.09963417 -0.04047966  0.00589013 -0.01581055  0.17780161 -0.646030

# Try using Cholesky Factorization

In [532]:
# print(tf.linalg.det(H))
# H_sparse = tf.sparse.from_dense(H)
H_chol = tf.linalg.cholesky(H)
# print(H_chol)

tf.Tensor(
[[nan  0.  0. ...  0.  0.  0.]
 [nan nan  0. ...  0.  0.  0.]
 [nan nan nan ...  0.  0.  0.]
 ...
 [nan nan nan ... nan  0.  0.]
 [nan nan nan ... nan nan  0.]
 [nan nan nan ... nan nan nan]], shape=(42, 42), dtype=float32)


2023-01-18 15:36:21.314356: W tensorflow/core/kernels/linalg/cholesky_op_gpu.cu.cc:211] Cholesky decomposition was not successful for batch 0. The input might not be valid. Filling lower-triangular output with NaNs.


In [541]:
#getting weird numerical errors here making H very slightly non-symmetric
#  preventing me from using cholesky decomp.
H_ij = J[0].numpy().T @ tf.cast(omega[0], tf.float32) @ J[0]
# print(H_ij)
test = H_ij.numpy() - H_ij.numpy().T
# print(test)

print(H, "\n")
# test = np.tril(H.numpy())
test = np.tril(H.numpy(), -1)
# print(test)
print(np.tril(H.numpy()).T + np.tril(H.numpy(),-1) )

tf.Tensor(
[[2.1119098e+05 1.1033832e+05 2.0050373e+04 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.1033832e+05 9.8016703e+04 1.1668230e+04 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.0050375e+04 1.1668232e+04 1.9388010e+03 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.8292395e+01
  2.8159603e+01 7.5015114e+01]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 2.8161198e+01
  4.3370384e+01 1.1544238e+02]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 7.5017174e+01
  1.1543901e+02 3.0748639e+02]], shape=(42, 42), dtype=float32) 

[[2.1119098e+05 1.1033832e+05 2.0050375e+04 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.1033832e+05 9.8016703e+04 1.1668232e+04 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.0050375e+04 1.1668232e+04 1.9388010e+03 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.8292395e+01
  2.8161198e+01 7.5017174e+01]
 [0.0000000e+0

In [542]:
# test = H.numpy() - H.numpy().T
h = np.random.randn(3,5)
ohm = np.eye(3)
test = h.T @ ohm @ h
print(test)

[[ 3.75665171  0.78445855  0.47290638  0.84413965 -1.1716865 ]
 [ 0.78445855  1.04991795 -0.04214071  0.94974895 -0.55332488]
 [ 0.47290638 -0.04214071  0.09245641 -0.14894456 -0.07675058]
 [ 0.84413965  0.94974895 -0.14894456  2.52638372 -0.80502034]
 [-1.1716865  -0.55332488 -0.07675058 -0.80502034  0.51758776]]


In [543]:
#test
indices = tf.constant([[1,3], [3,1], [5,5]], dtype = tf.int64) #[N, ndims]
values = tf.constant([1.5, 1.5, -0.55])
dense_shape = tf.constant([6, 6], dtype = tf.int64)
sparse_J =  tf.sparse.SparseTensor(indices, values, dense_shape)
# print(sparse_J)
print(tf.sparse.to_dense(sparse_J))
# print(tf.linalg.det(tf.sparse.to_dense(sparse_J)))

test = tf.sparse.to_dense(sparse_J)[None,:,:] + 10*tf.eye(6)[None,:,:]
print(test)

chol = tf.linalg.cholesky(test) #needs to be symmetric and positive definite
print(chol)

rhs = tf.random.normal([1,6,6])
ans =  tf.linalg.cholesky_solve(chol, rhs)
print(ans)

tf.Tensor(
[[ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    1.5   0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.5   0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.55]], shape=(6, 6), dtype=float32)
tf.Tensor(
[[[10.    0.    0.    0.    0.    0.  ]
  [ 0.   10.    0.    1.5   0.    0.  ]
  [ 0.    0.   10.    0.    0.    0.  ]
  [ 0.    1.5   0.   10.    0.    0.  ]
  [ 0.    0.    0.    0.   10.    0.  ]
  [ 0.    0.    0.    0.    0.    9.45]]], shape=(1, 6, 6), dtype=float32)
tf.Tensor(
[[[3.1622777  0.         0.         0.         0.         0.        ]
  [0.         3.1622777  0.         0.         0.         0.        ]
  [0.         0.         3.1622777  0.         0.         0.        ]
  [0.         0.47434163 0.         3.1264994  0.         0.        ]
  [0.         0.         0.         0.         3.1622777  0.        ]
  [0.         0.         0.         0.         0.         3.0740852 ]]], shap

Similar to ICET, we can use Newton-Raphson to iteratively solve for small perterbations to the state vector $x$ that bring us towards a better solution

\begin{equation}
\Large
x \rightarrow x + \Delta x
\end{equation}

# TODO

Figure out references in Jupyter Notebook


# Questions

Should $A_{ij} = -B_{ij}$?

Are there advantages to reporting state vector in unit quaternions? Besides being less intuitive to work with I've read they can also introduce problems in optimization routine since they add an additional degree of freedom. Every guide I've seen so far uses homogenous coordinate representation for the transformation matrices when computing the loss function, since they can be applied neatly in series. So rotation matrices do not produce singularities?

Is what we did in ICET to handle sparsity (i.e. accumulating contributions for corresponding elements rather than performing full matrix inversion) similar to a Choelesky Decomposition?

<span style="color:red"> What is going on with the numerics in get_e_ij()? Why is it not just t2v(Zij) - t2v(Zij)?? </span>



# Potential Contributions

Use ICET error covaraince estimation to demonstrate improvement in accuracy for Volpe Dataset

Use ICET output to inform $\Omega$ as a spatially dependant weighting field rather than simply indexing each unique point. Explore Gaussian Process Regression?


In [ ]:
test = tf.random.normal([600,600])
# print(tf.linalg.pinv(test))

https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.inv.html#scipy.sparse.linalg.inv

In [ ]:
st_a = tf.sparse.SparseTensor(indices=[[0, 2], [3, 4]],
                       values=[31.0, 2.0], 
                       dense_shape=[4, 6])
st_b = tf.sparse.SparseTensor(indices=[[0, 2], [3, 0]],
                       values=[56.0, 38.0],
                       dense_shape=[4, 6])
st_sum = tf.sparse.add(st_a, st_b)
# print(st_sum)
print(tf.linalg.pinv(tf.sparse.to_dense(st_sum)).numpy())